# Stable Baselines3 튜토리얼 - Getting Started

이 튜토리얼은 Stable Baselins3 라이브러리의 공식 튜토리얼과 예제 코드를 참고/번역하여 작성되었습니다.

출처 : [https://github.com/Stable-Baselines-Team/rl-colab-notebooks](https://github.com/Stable-Baselines-Team/rl-colab-notebooks)

Stable-Baselines3 Github: https://github.com/DLR-RM/stable-baselines3

공식 문서: https://stable-baselines.readthedocs.io/en/master/

## 소개

Stable Baselines3는 OpenAI의 Baselines 라이브러리를 기반으로 개발된 강화학습 라이브러리입니다.

A2C, DDPG, DQN, HER, PPO, SAC TD3 등의 알고리즘이 구현되어 있습니다.

## 설치

```bash
pip install "stable-baselines3[extra]"
```

## Import

Stable-Baselines3는 OpenAI Gymnasium 환경에서 사용할 수 있습니다.

In [1]:
import stable_baselines3
import gymnasium as gym
import numpy as np

print(f"{stable_baselines3.__version__=}")
print(f"{gym.__version__=}")

stable_baselines3.__version__='2.3.2'
gym.__version__='0.29.1'


### 알고리즘 import

다음과 같이 필요한 알고리즘을 import 할 수 있습니다.


In [2]:
from stable_baselines3 import PPO

### policy import

SB3에서는 알고리즘과 policy가 독립적으로 구현되어있습니다.

policy는 모델의 구조를 만ㄷ는데 사용됩니다.

알고리즘의 종류에 따라 사용가능한 policy가 조금씩 다릅니다.

e.g.)
- `MlpPolicy` : feature extraction network이 MLP인 경우 사용
- `CnnPolicy` : feature extraction network이 CNN인 경우 사용
- `MultiInputPolicy` : multiple input을 받는 경우 (`dict` 형태로 여러 input을 받는 경우)

사실 `SAC`의 경우, 별도의 `MlpPolicy`가 구현되어 있습니다.

따라서, policy를 import하기 보단 string으로 policy를 지정하는 것이 좋습니다.

`PPO(MlpPolicy, env)` -> `PPO("MlpPolicy", env, verbose=1)`




In [3]:
from stable_baselines3.ppo import MlpPolicy

## Gymnaisum 환경 및 agent 생성

이 예제에서는 `CartPole-v1` 환경을 사용합니다.

CartPole 환경 : [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)


In [4]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
model = PPO(MlpPolicy, env, verbose=0, gamma=0.90)

우선, 훈련되지 않은 agent를 평가해보겠습니다.

`evaluate_policy` 함수를 사용하여 agent를 평가할 수 있습니다.

SB3에는 이와 같은 유용한 함수들이 많이 구현되어 있습니다.

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
# evaluation 환경은 training 환경과 별도로 구성하는 것이 좋습니다.
eval_env = gym.make("CartPole-v1", render_mode="rgb_array")

# 훈련을 하지 않았으므로 random policy일 것입니다.
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

# 100개의 evaluation episode를 실행했을 때, 평균 보상과 표준편차를 출력합니다.
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/chiyeong/Documents/projects/Mentoring-RL-Env/.venv/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:94.04 +/- 77.94


## agent 환경 상호작용

In [7]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

In [8]:
env_id = "CartPole-v1"
video_dir = "videos/"
video_length = 500

vec_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

obs = vec_env.reset()

# Record the video starting at the first step
vec_env = VecVideoRecorder(vec_env, video_dir,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix=f"random-agent-{env_id}")

obs = vec_env.reset()
for _ in range(video_length + 1):
    action, _ = model.predict(obs)
    obs, _, _, _ = vec_env.step(action)

# Save the video
vec_env.close()

Saving video to /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/random-agent-CartPole-v1-step-0-to-step-500.mp4
Moviepy - Building video /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/random-agent-CartPole-v1-step-0-to-step-500.mp4.
Moviepy - Writing video /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/random-agent-CartPole-v1-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/random-agent-CartPole-v1-step-0-to-step-500.mp4


In [9]:
from IPython.display import Video
import os

# Path to your video file
video_path = f"{video_dir}random-agent-{env_id}-step-0-to-step-{video_length}.mp4"


if os.path.exists(video_path):
    # Display the video
    display(Video(video_path, embed=True))
else:
    print(f"Video file not found: {video_path}")

## agent 학습, 평가

`model.learn()` 함수를 사용하여 간단하게 agent를 학습시킬 수 있습니다.

In [10]:
model.learn(total_timesteps=10_000)

In [11]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/chiyeong/Documents/projects/Mentoring-RL-Env/.venv/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:418.18 +/- 102.74


In [12]:
env_id = "CartPole-v1"
video_dir = "videos/"
video_length = 500

vec_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

obs = vec_env.reset()

# Record the video starting at the first step
vec_env = VecVideoRecorder(vec_env, video_dir,
                           record_video_trigger=lambda x: x == 0, video_length=video_length,
                           name_prefix=f"PPO-{env_id}")

obs = vec_env.reset()
for _ in range(video_length + 1):
    action, _ = model.predict(obs)
    obs, _, _, _ = vec_env.step(action)

# Save the video
vec_env.close()

Saving video to /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/PPO-CartPole-v1-step-0-to-step-500.mp4
Moviepy - Building video /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/PPO-CartPole-v1-step-0-to-step-500.mp4.
Moviepy - Writing video /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/PPO-CartPole-v1-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /Users/chiyeong/Documents/projects/Mentoring-RL-Env/exmaples/videos/PPO-CartPole-v1-step-0-to-step-500.mp4


In [13]:
# Path to your video file
video_path = f"{video_dir}PPO-{env_id}-step-0-to-step-{video_length}.mp4"


if os.path.exists(video_path):
    # Display the video
    display(Video(video_path, embed=True))
else:
    print(f"Video file not found: {video_path}")

## 모델 저장 및 불러오기

SB3는 학습된 모델을 저장하고 불러오는 기능을 제공합니다.

- `model.save("model_name")`
- `model = model.load("model_name")`

In [14]:
import os

save_dir = "tmp/models/"

# PPO_tutorial.zip 파일로 모델을 저장합니다.
model.save(f"{save_dir}/PPO_tutorial")

# 모델이 정상적으로 저장됐는지 확인하기 위해, observation을 샘플링하고 예측을 해봅니다.
obs = [model.env.observation_space.sample() for _ in range(20)]

print("pre saved : ", model.predict(obs, deterministic=True)[0])

# del model  # 현재 메모리에 있는 모델을 삭제합니다.

# 저장된 모델을 불러옵니다.
loaded_model = PPO.load(f"{save_dir}/PPO_tutorial")
# 불러온 모델이 같은 observation에 대해 같은 예측을 하는지 확인합니다.
print("loaded".ljust(9) + " : ", loaded_model.predict(obs, deterministic=True)[0])

pre saved :  [0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0]
loaded    :  [0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0]


모델은 하이퍼파라미터와 함께 저장됩니다.

In [15]:

# show the save hyperparameters
print(f"saved  : gamma={model.gamma}, n_steps={model.n_steps}")
print(f"loaded : gamma={loaded_model.gamma}, n_steps={loaded_model.n_steps}")

saved  : gamma=0.9, n_steps=2048
loaded : gamma=0.9, n_steps=2048
